### Analyzing MEPS-HC Data with SAS® 9.4 M6 
#### By Pradip K. Muhuri, PhD

## Exercise 3

### Objective

* Estimate mean out-of-pocket health care expenses for individuals who were aged 26-30 years with high income and uninsured for the whole year

### Data and Analysis
* Combine data from 2016 and 2017 MEPS Full-Year Consolidated Files 
* Run PROC FREQ and PROC MEANS for data checks
* Run PROC SURVEYMEANS for complex survey estimates


### Concatenate two full-year data files into one SAS data set
* KEEP only needed variables when loading the original SAS data sets
* RENAME year-specific variables as variable with year suffix 
        prior to concatenating those files
* Create new variables including the subpopulation variable

In [ ]:
options nodate nonumber nosource;
ods html close;
LIBNAME CDATA 'C:\DATA';
DATA POOL;
    SET CDATA.H192 
       (in=a KEEP= DUPERSID INSCOV16 PERWT16F VARSTR VARPSU POVCAT16 AGELAST TOTSLF16
        RENAME=(INSCOV16=INSCOV PERWT16F=PERWT POVCAT16=POVCAT TOTSLF16=TOTSLF))
        
       CDATA.H201 
        (in=b KEEP= DUPERSID INSCOV17 PERWT17F VARSTR VARPSU POVCAT17 AGELAST TOTSLF17
        RENAME=(INSCOV17=INSCOV PERWT17F=PERWT POVCAT17=POVCAT TOTSLF17=TOTSLF));
                
     /* Create a YEAR Variable for checking data*/
     if a=1 then year=2016;
     else if b=1 then year=2017;

     POOLWT = PERWT/2 ;  /* Pooled survey weight */

     /*Create a dichotomous SUBPOP variable 
     (POPULATION WITH AGE=26-30, UNINSURED WHOLE YEAR, AND HIGH INCOME)
     */
     IF 26 LE AGELAST LE 30 AND POVCAT=5 AND INSCOV=3 THEN SUBPOP=1;
     ELSE SUBPOP=2;    
 run;


### Code snippet for PROC FORMAT

In [ ]:
options nocenter nodate nonumber nosource;
PROC FORMAT;
VALUE POVCAT 
    1 = '1 POOR/NEGATIVE'
    2 = '2 NEAR POOR'
    3 = '3 LOW INCOME'
    4 = '4 MIDDLE INCOME'
    5 = '5 HIGH INCOME'   ;

VALUE INSF
    1 = '1 ANY PRIVATE'
    2 = '2 PUBLIC ONLY'
    3 = '3 UNINSURED';

   VALUE  SUBPOP (max= 50)
    1 = 'AGE 26-30, UNINSURED WHOLE YEAR, AND HIGH INCOME'
    2 ='OTHERS';
run;


#### Use PROC FREQ and PROC MEANS for data checks

In [ ]:
TITLE "COMBINED MEPS DATA FROM 2016 and 2017 Consolidated Files";
PROC FREQ DATA=WORK.POOL;
    TABLES YEAR*SUBPOP*POVCAT*INSCOV /LIST MISSING nopercent ;
           FORMAT SUBPOP SUBPOP. POVCAT POVCAT. INSCOV INSF.;
RUN;
PROC MEANS DATA=POOL;
RUN;

#### Code explanation for the next cell
* With no statistic-keywords specifiedin the PROC SURVEYMEANS statement, it computes the NOBS, MEANS, STDERR, and CLM statistics by default.

* ODS GRAPHICS OFF;  - suppresses the graphics 
* ODS EXCLUDE STATISTICS; - tells SAS not to generate output for the overall population 

* PROC SURVEYMEANS estimates mean out-of-pocket health care expenses for individuals who were aged 26-30 years with high income and uninsured for the whole year


In [ ]:
TITLE2 'WEIGHTED ESTIMATE FOR OUT-OF-POCKET EXPENSES FOR PERSONS AGES 26-30, UNINSURED WHOLE YEAR, AND HIGH INCOME';
ODS GRAPHICS OFF;
ODS EXCLUDE STATISTICS; 
PROC SURVEYMEANS DATA=WORK.POOL; 
    VAR  TOTSLF;
    STRATUM VARSTR ;
    CLUSTER VARPSU ;
    WEIGHT  POOLWT;
  DOMAIN  SUBPOP("AGE 26-30, UNINSURED WHOLE YEAR, AND HIGH INCOME");
    FORMAT SUBPOP SUBPOP.;
RUN;